In [2]:
import numpy as np
import pandas as pd
import wrds
import pickle
from scipy.stats.mstats import winsorize
from tqdm import tqdm

In [3]:
tmp_data_path =  '../MA_data/data/tmp'
data_path = '../MA_data/data'
s_year = 1997-1
e_year = 2020

In [113]:
def intepolation(df):
    '''
    first, interpolate using historical data for specific firms(not global)
    after this, if the variable still has mising ratio > 10%, drop
    finally, interpolate using global mean
    
    '''
    # some helpers
    def interpolate_group_mean(df):
        '''
        first two columns are not fv; they are: gvkey, year

        '''
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        for colname in tqdm(df.columns[2:]):
            # impute na with mean of itself
            df[colname] = df.groupby("gvkey")[colname].transform(lambda x: x.fillna(x.mean()))
        return df
    def print_missing_ratio(df):
        print(df.isna().sum()/df.shape[0])

    def intepolate_global_mean(df):
        df = df.reset_index(drop = True)
        for colname in df[2:]:
            df[colname].fillna(value=df[colname].mean(skipna=True), inplace=True)
        return df
    
    
    
    
    
    def __main__(df):
        
        print("original missing ratios: ")
        print_missing_ratio(df)

        df = interpolate_group_mean(df)
        print("after group interpolation, missing ratios: ")
        print_missing_ratio(df)

        helper = pd.DataFrame(df.isna().sum()/df.shape[0]).reset_index(drop=False)
        helper.columns = ['vars', 'missing_ratio']

        keep_colums = helper[helper.missing_ratio <= 0.1].vars.values.tolist()
        print("drop those columns:", helper[helper.missing_ratio > 0.1].vars.values.tolist())
        df = df[keep_colums] # only keep those columns


        helper.columns = ['vars', 'missing_ratio']
        df = intepolate_global_mean(df)
        print("after global interpolation, missing ratios: ")
        print_missing_ratio(df)    

        #df.to_pickle(tmp_data_path+"/full_fv.pickle")
        return df

    df = df.copy()

    df = __main__(df)
    return df
    

# create ratios


run [Appendix 2.1](./Appendix_2.1_prepare_fin_vars.ipynb)

# Dealing with missing values
- for those has at least one year data, using the last time data to intepolate
- just use mean to interpolate all missing values
    - may be intepolate before making ratios are a better solution... at least part of the number is correct

In [109]:
df_fv_raw = pd.read_pickle(f'{tmp_data_path}/fv_raw_{s_year}_{e_year}.pickle')

In [117]:
with open(tmp_data_path+f"/tnic_info_3_pairs_{1996}_{2019}", 'rb') as f:
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

In [137]:
df_lst = []
for year in range(1996, 2019+1):
    df_fv_year_gvkey = df_fv_raw[df_fv_raw.gvkey.isin(gvkey_lsts[year]) & (df_fv_raw.year == year)]
    df_lst.append(df_fv_year_gvkey)

In [138]:
df_fv_raw_2 = pd.concat(df_lst)

In [141]:
tmp_df = intepolation(df_fv_raw_2)

original missing ratios: 
gvkey         0.000000
year          0.000000
at            0.244232
sale          0.266473
ch            0.700699
rdip          0.890508
m2b           0.788636
lev           0.692813
roa           0.252675
ppe           0.315130
cash2asset    0.700788
cash2sale     0.745760
sale2asset    0.293834
cr            0.836334
gsi           0.746518
de            0.699481
roe           0.697433
d_sale        0.423453
d_at          0.364457
dtype: float64


100%|██████████| 17/17 [00:38<00:00,  2.29s/it]


after group interpolation, missing ratios: 
gvkey         0.000000
year          0.000000
at            0.006524
sale          0.004854
ch            0.239039
rdip          0.513559
m2b           0.286705
lev           0.233499
roa           0.007120
ppe           0.020152
cash2asset    0.239063
cash2sale     0.257110
sale2asset    0.008725
cr            0.426187
gsi           0.409873
de            0.235902
roe           0.236088
d_sale        0.037401
d_at          0.023143
dtype: float64
drop those columns: ['rdip', 'cr', 'gsi']
after global interpolation, missing ratios: 
gvkey         0.0
year          0.0
at            0.0
sale          0.0
ch            0.0
rdip          0.0
m2b           0.0
lev           0.0
roa           0.0
ppe           0.0
cash2asset    0.0
cash2sale     0.0
sale2asset    0.0
cr            0.0
gsi           0.0
de            0.0
roe           0.0
d_sale        0.0
d_at          0.0
dtype: float64


In [143]:
with open(data_path+"/fv_raw_full_1996_2019.pickle", 'wb') as f:
    pickle.dump(tmp_df, f)

In [147]:
check = np.unique(tmp_df[tmp_df.year == 1996].gvkey.values).tolist()

In [148]:
len(check)

7528

## Deal with NA, but only for frequent acquirers
- watch out that you don't necesserily need the fv of focal firms every year!

In [106]:
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_{s_year+1}_{e_year}")

In [94]:
A_freq = pd.DataFrame(sdc_tnic.AGVKEY.value_counts()).reset_index(drop=False)
A_freq = A_freq[A_freq.AGVKEY >= 5]
A_freq.columns = ["GVKEY", "freq"]
print(f"totally {A_freq.shape[0]} numbers of frequent Acquirers")

totally 511 numbers of frequent Acquirers


In [104]:
full_ratios = intepolation(ratios_raw_afreq)

original missing ratios: 
gvkey         0.000000
year          0.000000
at            0.000000
sale          0.000365
ch            0.038560
rdip          0.243482
m2b           0.089335
lev           0.029170
roa           0.000000
ppe           0.008113
cash2asset    0.038560
cash2sale     0.041021
sale2asset    0.000365
cr            0.287147
gsi           0.222425
de            0.030356
roe           0.030356
d_sale        0.007657
d_at          0.000000
dtype: float64


100%|██████████| 17/17 [00:01<00:00, 13.47it/s]

after group interpolation, missing ratios: 
gvkey         0.000000
year          0.000000
at            0.000000
sale          0.000365
ch            0.038560
rdip          0.243482
m2b           0.089335
lev           0.029170
roa           0.000000
ppe           0.008113
cash2asset    0.038560
cash2sale     0.041021
sale2asset    0.000365
cr            0.287147
gsi           0.222425
de            0.030356
roe           0.030356
d_sale        0.007657
d_at          0.000000
dtype: float64
drop those columns: ['rdip', 'cr', 'gsi']
after global interpolation, missing ratios: 
gvkey         0.0
year          0.0
at            0.0
sale          0.0
ch            0.0
m2b           0.0
lev           0.0
roa           0.0
ppe           0.0
cash2asset    0.0
cash2sale     0.0
sale2asset    0.0
de            0.0
roe           0.0
d_sale        0.0
d_at          0.0
dtype: float64


In [1]:
full_ratios

NameError: name 'full_ratios' is not defined

# normalize every columns by:


$$z_{i}=\frac{x_{i}-\min (x)}{\max (x)-\min (x)}$$

In [4]:
with open(tmp_data_path+"/afreq_full_fv.pickle", "rb") as f:
    fv = pickle.load(f)

In [31]:
new_fv = pd.concat([fv.iloc[:,:2], N01_normalize(fv.iloc[:, 2:])], axis=1)

In [32]:
fv.shape

(10970, 16)

In [33]:
new_fv.shape

(10970, 16)

In [34]:
new_fv[:10]

,gvkey,year,at,sale,ch,m2b,lev,roa,ppe,cash2asset,cash2sale,sale2asset,de,roe,d_sale,d_at
0,10000,1996,-0.278443,-0.461856,-0.248260,-0.504117,0.719369,-0.010192,-0.106231,-0.694390,-0.371059,0.845787,-0.030604,-0.075038,-0.056590,-0.126479
1,10000,1997,-0.278727,-0.467743,-0.246081,-0.371932,0.675835,-0.253207,-0.044381,-0.432318,-0.327141,0.549244,-0.014330,-0.184290,-0.173109,-0.205551
2,10000,1998,-0.279059,-0.464487,-0.244889,-0.360806,0.118326,0.167571,-0.079297,-0.240541,-0.315782,0.989718,-0.086256,-0.012041,0.011778,-0.219277
3,10000,1999,-0.278853,-0.464166,-0.241853,-0.465665,0.350748,-0.082727,-0.152754,0.093981,-0.268768,0.891562,-0.061353,-0.108727,-0.058656,-0.107886
4,10000,2000,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479
5,10000,2001,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479
6,10000,2002,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479
7,10000,2003,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479
8,10000,2004,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479
9,10000,2005,-0.278061,-0.454646,-0.245271,-0.425630,0.466069,0.347284,-0.369497,-0.318317,-0.320687,1.144396,-0.048136,-0.095024,-0.056590,-0.126479


In [15]:
df = np.array(fv.copy().iloc[:, 2:])

In [16]:
def N01_normalize(df):
    '''
    df could be df or array
    '''
    df = df.copy()
    normalized_df=(df-df.mean())/df.std()
    return normalized_df
    
    

In [18]:
N01_normalize(df)

array([[-0.08097418, -0.07891585, -0.09413372, ..., -0.0942313 ,
        -0.09423232, -0.09423187],
       [-0.08198573, -0.08235314, -0.093876  , ..., -0.09423288,
        -0.09423746, -0.09423431],
       [-0.08316517, -0.08045187, -0.09373493, ..., -0.09423039,
        -0.0942293 , -0.09423474],
       ...,
       [11.1920295 ,  3.53028412, -0.06740071, ..., -0.09422907,
        -0.09423137, -0.0942285 ],
       [11.61229562,  3.75224745, -0.06740071, ..., -0.09422907,
        -0.0942314 , -0.0942319 ],
       [11.0625226 ,  3.55054942, -0.06740071, ..., -0.09422907,
        -0.09423381, -0.09423369]])

In [17]:
normalized_df.shape

(10970, 14)